## Global Spatial Autocorrelation

In [2]:
# Graphics
import matplotlib.pyplot as plt
import seaborn
from pysal.viz import splot
from splot.esda import plot_moran
import contextily

import shapely

# Analysis
import geopandas
import pandas
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed

- The notion of spatial autocorrelation relates to the existence of a “functional relationship between what happens at one point in space and what happens elsewhere”

- Spatial autocorrelation thus has to do with the degree to which the similarity in values between observations in a dataset is related to the similarity in locations of such observations.

- Like a traditional correlation

- Spatial autocorrelation relates the value of the variable of interest in a given location, with values of the same variable in other locations.

### Understanding Spatial Autocorrelation